## Steven Smith, King Ting Helen Wong, Zeqing Yan
## Professor Abu Kamruzzaman
## Programming for Analytics (CIS 3120-NFA)
## 20 November 2020                  

# <h3 align="center">Yahoo Finance Overview</h3> 

In [ ]:
#importing libraries
from lxml import html
import requests
import json
import argparse
from collections import OrderedDict

In [ ]:
#Fetching HTML title
def get_headers():
    return {"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
            "accept-encoding": "gzip, deflate, br",
            "accept-language": "en-GB,en;q=0.9,en-US;q=0.8,ml;q=0.7",
            "cache-control": "max-age=0",
            "dnt": "1",
            "sec-fetch-dest": "document",
            "sec-fetch-mode": "navigate",
            "sec-fetch-site": "none",
            "sec-fetch-user": "?1",
            "upgrade-insecure-requests": "1",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.122 Safari/537.36"}

In [ ]:
#Going through contents, specific to the company we are scrapping financial data for
def parse(ticker):
    url = "http://finance.yahoo.com/quote/%s?p=%s" % (ticker, ticker)
    response = requests.get(
        url, verify=False, headers=get_headers(), timeout=30)
    print("Parsing %s" % (url))
    parser = html.fromstring(response.text)
    summary_table = parser.xpath(
        '//div[contains(@data-test,"summary-table")]//tr')
    summary_data = OrderedDict()
    other_details_json_link = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com".format(
        ticker)
    summary_json_response = requests.get(other_details_json_link)

In [ ]:
#fetching specific financial data that we want such as target mean price and earnings
    try:
        json_loaded_summary = json.loads(summary_json_response.text)
        summary = json_loaded_summary["quoteSummary"]["result"][0]
        y_Target_Est = summary["financialData"]["targetMeanPrice"]['raw']
        earnings_list = summary["calendarEvents"]['earnings']
        eps = summary["defaultKeyStatistics"]["trailingEps"]['raw']
        datelist = []

        for i in earnings_list['earningsDate']:
            datelist.append(i['fmt'])
        earnings_date = ' to '.join(datelist)

        for table_data in summary_table:
            raw_table_key = table_data.xpath(
                './/td[1]//text()')
            raw_table_value = table_data.xpath(
                './/td[2]//text()')
            table_key = ''.join(raw_table_key).strip()
            table_value = ''.join(raw_table_value).strip()
            summary_data.update({table_key: table_value})
        summary_data.update({'1y Target Est': y_Target_Est, 'EPS (TTM)': eps,'Earnings Date': earnings_date, 'ticker': ticker,'url': url})
                return summary_data
    except ValueError:
        print("Failed to parse json response")
        return {"error": "Failed to parse json response"}
    except:
        return {"error": "Unhandled Error"}

In [ ]:
if __name__ == "__main__":
    argparser = argparse.ArgumentParser()
    argparser.add_argument('ticker', help='')
    args = argparser.parse_args()
    ticker = args.ticker
    print("Fetching data for %s" % (ticker))
    scraped_data = parse(ticker)
    print("Writing data to output file")
    with open('%s-summary.json' % (ticker), 'w') as fp:
        json.dump(scraped_data, fp, indent=4)

In [ ]:
#Here we are using numpy as well as panda to compute the sharpe ratio, a financial ratio that measures an investment's return compare to its risk, over certain number of days.
import numpy as np
import pandas as pd

In [ ]:
# Simulate sharpe ratio of cumulative returns of 100 days 
N = 100
R = pd.DataFrame(np.random.normal(size=100)).cumsum()

In [ ]:
r = (R - R.shift(1))/R.shift(1)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

source = requests.get('https://finance.yahoo.com/quote/AMZN/key-statistics?p=AMZN').text

soup = BeautifulSoup(source,'lxml')

In [ ]:
table = soup.find('div',class_='M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)')

tr = table.find('tr',class_='Bdtw(0px) C($primaryColor)')

current = tr.find_all('span')[2].text
first = tr.find_all('span')[3].text
second = tr.find_all('span')[4].text
third = tr.find_all('span')[5].text 
fourth = tr.find_all('span')[6].text

print(current,first,second,third,fourth)

tbody = table.find('tbody')

for tr in tbody.find_all('tr'):
    data_type = tr.find_all('span')[0].text
    current = tr.find_all('td')[1].text
    first = tr.find_all('td')[2].text
    second = tr.find_all('td')[3].text
    third = tr.find_all('td')[4].text
    fourth = tr.find_all('td')[5].text
    print(data_type,current,first,second,third,fourth)
    
print()

In [ ]:
# writing/saving as csv file
df = pd.DataFrame({'Amazondata':['Market Cap (intraday)','Enterprise Value','Trailing P/E','Forward P/E','PEG Ratio (5 yr expected)','Price/Sales','Price/Book','Enterprise Value/Revenue','Enterprise Value/EBITDA'],
                   'current':['1.59T','1.60T','122.08','58.14','1.19','5.00','21.58','4.96','39.12'],
                    '6/30/2020':[ '1.38T', '1.40T', '131.81',  '128.21', '2.97', '4.70', '21.18', '15.71', '112.89' ],
                    '3/31/2020':['972.91B','981.09B', '84.73', '66.23','1.50','3.50','15.68','13.00','107.26'] ,
                    '12/31/2019':['920.22B','936.35B','81.87','58.14','1.32','3.50','16.28','10.71','87.69'],
                   '9/30/2019':['859.28B','876.28B','72.00','44.84','1.02','3.46','16.19','12.52','102.00']})

                                  
df    
df.to_csv('Users/Jackie333/Desktop/data.csv')
df.isnull()
(df.isnull()).sum()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data_file = np.loadtxt('Users/Jackie333/Desktop/data.csv', delimiter=',')

time =data_file[:,0]

time = time - time[0]

sensors =data_file[:,1:5]

avg = np.mean(sensors,axis=1)

my_data = np.vstack((time,sensors.T,avg))
my_data = my_data.T
np.savetxt('export.csv',my_data,delimiter=',')

plt.plot(time/60,sensors[:,1],'ro')
plt.plot(time/60,avg,'b')
plt.legend(['Sensor 2','Average'], loc='best')
plt.xlabel('Time (min)')
plt.ylabel('Values')
plt.savefig('my_plot.png')
plt.show()

In [ ]:
import seaborn as sns
sns.set_style("whitegrid")

d = pd.date_range(start='1/1/2008', end='12/1/2015')
df = pd.DataFrame(d, columns=['Date'])
df['returns'] = np.random.rand(d.size, 1)
df = df.set_index('Date')

df['Users/Jackie333/Desktop/data.csv'] = df.returns.rolling(180).apply(lambda x: (x.mean() - 0.02) / x.std(), raw = True)
df.fillna(0, inplace = True)
df[df['Users/Jackie333/Desktop/data.csv'] > 0].rolling_SR.plot(style='-', lw=3, color='orange', label='Sharpe', figsize = (10,7))\.axhline(y = 1.6, color = "blue", lw = 3,linestyle = '--')

plt.ylabel('Sharpe ratio')
plt.legend(loc='best')
plt.title('Rolling Sharpe ratio (6-month)')
plt.show()